In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

e:\ML Projects\Autogen-AgentAI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)

True

In [3]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [4]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    "Send a push notification to the user"
    requests.post(pushover_url, data = {"token" : pushover_token, "user" : pushover_user, "message" : text})

tool_push = Tool(
    name = "send_push_notification",
    func = push,
    description="useful when you want to senf push notifications"
)

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
# !playwright install

In [8]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

# If you get a NotImplementedError here or later, see the Heads Up at the top of the notebook

async_browser =  create_async_playwright_browser(headless=False)  # headful mode
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

In [9]:
for tool in tools:
    print(f"{tool.name} = {tool}")

click_element = async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\ASUS\AppData\Local\ms-playwright\chromium-1181\chrome-win\chrome.exe> version=139.0.7258.5>
navigate_browser = async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\ASUS\AppData\Local\ms-playwright\chromium-1181\chrome-win\chrome.exe> version=139.0.7258.5>
previous_webpage = async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\ASUS\AppData\Local\ms-playwright\chromium-1181\chrome-win\chrome.exe> version=139.0.7258.5>
extract_text = async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\ASUS\AppData\Local\ms-playwright\chromium-1181\chrome-win\chrome.exe> version=139.0.7258.5>
extract_hyperlinks = async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\ASUS\AppData\Local\ms-playwright\chromium-1181\chrome-win\chrome.exe> version=139.0.7258.5>
get_elements = async_browser=<Browser type=<Browse